# Impute ATAC from RNA (Mouse brain dataset)

In this notebook, we used SpaMosaic to impute the missing ATAC peak assays in a postnatal mouse brain dataset. The dataset consists of three sections, all measured with both RNA and ATAC profiles. We removed the ATAC profiles from the first section, trained SpaMosaic on the modified dataset, and then imputed the ATAC peak data for the first section based on RNA. 

Data used in this notebook can be downloaded from [google drive](https://drive.google.com/drive/folders/1GyOvHxweRYrq8Hiq5OdKhfSowUfcMoXY?usp=drive_link).

In [1]:
import os
import scanpy as sc
import scipy.sparse as sps
from os.path import join

import sys
sys.path.insert(0, '..')

from spamosaic.framework import SpaMosaic

To use the mclust clustering algorithm, we manually set the `R` and `rpy2` path here.

In [2]:
os.environ['R_HOME'] = '/disco_500t/xuhua/miniforge3/envs/Seurat5/lib/R'
os.environ['R_USER'] = '/disco_500t/xuhua/miniforge3/envs/Seurat5/lib/python3.8/site-packages/rpy2'
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # for CuBLAS operation and you have CUDA >= 10.2
import spamosaic.utils as utls
from spamosaic.preprocessing import RNA_preprocess, ADT_preprocess, Epigenome_preprocess
import spamosaic.metrics as eval

In [3]:
ad1_rna  = sc.read_h5ad('./s1_adata_rna.h5ad')
ad1_atac = sc.read_h5ad('./s1_adata_atac.h5ad')
ad2_rna  = sc.read_h5ad('./s2_adata_rna.h5ad')
ad2_atac = sc.read_h5ad('./s2_adata_atac.h5ad')
ad3_rna  = sc.read_h5ad('./s3_adata_rna.h5ad')
ad3_atac = sc.read_h5ad('./s3_adata_atac.h5ad')

### 1st-fold cv (cross validation)

Similar to integration, SpaMosaic performs modality alignment first and then imputes the missing modality profiles based on the modal-aligned latent space. Also, SpaMosaic requires the input dataset in the following format:
``` Python
{
    'rna':      [adata1_rna, adata2_rna,   None,         adata4_rna, ...],
    'protein':  [adata1_adt, None,         adata3_adt,   None,       ...],
    'atac':     [None,       adata2_atac,  None,         None,       ...],
    'histone':  [None,       None      ,   adata3_hist,  None,       ...],
    ...
}
```

In the dictionary, each key represents a modality and each modality key corresponds to list of `anndata` objects. Each `anndata` object contains modality-specific information for a particular section. For example, the first object `adata1_rna` under the 'rna' key holds the RNA profiles for the first section, while the first object `adata1_adt` under the 'protein' key holds protein profiles for the same section. If a section is not measured for a particular modality, its value in the list is `None`. For instance, the first element under the 'atac' and 'histone' keys is `None`, indicating that the first section was not measured with ATAC or histone modality. All lists have the same length, which corresponds to the number of sections in the target dataset.


SpaMosaic requires the modalities in a mosaic dataset to be directly or indirectly connected through one or multiple sections. If a pair of modalities occur in the same section, there is a direct connection between this pair of modalities, while an indirect connection requires multiple intermediary direct connections. 

SpaMosaic will automatically impute all the missing profiles in the input. 

In [4]:
input_dict = {
    'rna':  [ad1_rna, ad2_rna,  ad3_rna],
    'atac': [None,    ad2_atac, ad3_atac]
}

input_key = 'dimred_bc'

In [5]:
RNA_preprocess(input_dict['rna'], batch_corr=True, n_hvg=5000, batch_key='src', key=input_key)
hvp_name, hvp_idx = Epigenome_preprocess(input_dict['atac'], batch_corr=True, n_peak=50000, batch_key='src', key=input_key, return_hvf=True)

Use GPU mode.
	Initialization is completed.
	Completed 1 / 10 iteration(s).
	Completed 2 / 10 iteration(s).
	Completed 3 / 10 iteration(s).
Reach convergence after 3 iteration(s).
Use GPU mode.
	Initialization is completed.
	Completed 1 / 10 iteration(s).
	Completed 2 / 10 iteration(s).
	Completed 3 / 10 iteration(s).
	Completed 4 / 10 iteration(s).
Reach convergence after 4 iteration(s).


### training

SpaMosaic employs modality-specific graph neural networks to embed each modality's input into latent space. In mosaic integration, a section may contain one or multiple modalities, resulting in one or multiple sets of embeddings per section."

The crucial parameters include:
- `intra_knn`: how many nearest neighbors to consider when searching for spatial neighbors within the same section
- `inter_knn`: how many nearest neighbors to consider when searching for mutual nearest neighbors between sections
- `w_g`: the weight for spatial-adjacency graph, `1-w_g` is the weight for expression-adjacency graph

for training:
- `net`: which graph neural network to use (only support wlgcn now)
- `lr`: learning rate
- `T`: temperature parameter for contrastive learning
- `n_epochs`: number of training epochs

In [6]:
model = SpaMosaic(
    modBatch_dict=input_dict, input_key=input_key,
    batch_key='src', intra_knn=10, inter_knn=10, w_g=0.8, 
    seed=1234, 
    device='cuda:0'
)

model.train(net='wlgcn', lr=0.01, T=0.01, n_epochs=100)

batch0: ['rna']
batch1: ['rna', 'atac']
batch2: ['rna', 'atac']
------Calculating spatial graph...
The graph contains 23720 edges, 2372 cells.
10.0000 neighbors per cell on average.
------Calculating spatial graph...
The graph contains 24970 edges, 2497 cells.
10.0000 neighbors per cell on average.
------Calculating spatial graph...
The graph contains 92150 edges, 9215 cells.
10.0000 neighbors per cell on average.
------Calculating spatial graph...
The graph contains 24970 edges, 2497 cells.
10.0000 neighbors per cell on average.
------Calculating spatial graph...
The graph contains 92150 edges, 9215 cells.
10.0000 neighbors per cell on average.
Number of mnn pairs for rna:15799
Number of mnn pairs for atac:10046


100%|█████████████████████████████████████████| 100/100 [00:06<00:00, 16.33it/s]


### inference

After training, SpaMosaic can infer the modality-specific embedding for each section. These embeddings will be directly saved in original `anndata` objects. For example, the RNA-specific embeddings can be accessed by `ad1_rna.obsm['emb']`, `ad2_rna.obsm['emb']`, ... . The ATAC-specific embeddings can be accessed by `ad2_atac.obsm['emb']`, `ad3_atac.obsm['emb']`.

In [7]:
ad_embs = model.infer_emb(input_dict, emb_key='emb', final_latent_key='merged_emb')

### imputation

SpaMosaic employs a kNN-based strategy for imputation. One of its key advantages is that, after obtaining the modality-aligned latent space, SpaMosaic can directly impute multiple types of assays—such as peak counts, gene activity scores (GAS), and chromatin silence scores (CSS)—without the need to train multiple regression models.

Since SpaMosaic performs imputation by aggregating the measured profiles from other sections, we need to specify the reference assays in the `.layers` for each reference anndata object. For example, we aim to impute the missing peak assays for the first section and then we set the `.layers['counts']` as the raw peak counts data for `ad2_atac` and `ad3_atac`. If to impute the GAS, we need to transform the peak counts into GAS first for the sections 2 and 3, and set the `.layers['counts']` in `ad2_atac` and `ad3_atac` as corresponding GAS data. Note that even though RNA imputation is not required in this notebook, it is still necessary to specify the `.layers['counts']` for each rna anndata object. 

In [8]:
for mod, ads in input_dict.items():
    for ad in ads:
        if ad is not None:
            ad.layers['counts'] = ad.X.copy()  # set targeting layers

In [9]:
imputed_featureDict = model.impute(input_dict, emb_key='emb', layer_key='counts', imp_knn=10)

# format of imputed_featureDict
# {
#     'rna':  [None, None, None],
#     'atac': [array, None, None]
# }

impute atac-counts for batch-1


### evaluation

AUC without smoothing

In [10]:
gt_X = ad1_atac.X
pr_X = imputed_featureDict['atac'][0]  # take the imputed assay data

gt_X = gt_X.A if sps.issparse(gt_X) else gt_X
pr_X = pr_X.A if sps.issparse(pr_X) else pr_X
gt_X = gt_X[:, hvp_idx].copy()
pr_X = pr_X[:, hvp_idx].copy()

auc = eval.eval_AUC_all(gt_X, pr_X, bin_thr=1)  # binarize the target peak count (gt_X) with a threshold 1
auc

0.803555246513297

As the peak counts of ATAC data are noisy and sparse, its direct usage in performance assessment may result in underestimation of the imputation's efficacy. Therefore, we adopted the approach of [Tal et al., Nature Method. 2023](https://www.nature.com/articles/s41592-023-01909-9) in kNN smoothing on the raw peak count matrix. Specifically, we took the top 50 principal components of the expression data and computed the kNN graph (𝐾 = 50). Thereafter, we computed the average of the neighbors' expression values for each spot. The average values were then used to calculate the spot-spot/peak-peak PCC and AUROC metrics.

AUC with knn smoothing 

In [11]:
# dimension reduction
Epigenome_preprocess([ad1_atac], batch_corr=False, n_peak=50000, key='dimred')
# smoothing the peak matrix
smoothed_gt_X = eval.knn_smoothing(ad1_atac, hvf_name=hvp_name, dim_red_key='dimred', knn=50)

auc = eval.eval_AUC_all(smoothed_gt_X, pr_X, bin_thr=1)
auc

0.9839390106768231